In [14]:
from openai import OpenAI
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import gradio as gr


In [15]:
load_dotenv(override=True)
openai = OpenAI()

In [16]:
reader = PdfReader("../../1_foundations/me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [17]:
with open("../../1_foundations/me/summary.txt", "r", encoding='utf-8') as f:
    summary = f.read()

In [18]:
name = 'Ed Donner'

In [19]:
system_prompt = f"""
You are acting as a {name}. You're answering question on {name}'s website, particularly questions
related to {name}'s career, background, skills and experience. Your responsibility is to represent
{name} for interactions on the website as faithfully as possible. You're given a summary of {name}'s 
background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as
if talking to a potential client or future employer who came across. If you don't know the answer, say so.

LinkedIn profile contents: {linkedin}
Short {name}'s summary: {summary}
"""

In [20]:
def chat(message, history):
    messages = [{
        "role": "system",
        "content": system_prompt
    }] + history + [{
        "role": "user",
        "content": message
    }]
    
    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)
    return response.choices[0].message.content

In [8]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [9]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [21]:
evaluator_system_prompt = f"""
You are an evaluator that decides whether a response to a question is acceptable.
You are provided with a conversation betwen a User and an Agent. 
Your task is to decide whether the Agent's latest response is of acceptable quality.
The Agent is playing th role of {name} and is representing {name} on their website.
The Agent has been instructed to be professional and engaging, as if taking to a potential client or a future employer.
The Agent has been provided with context on {name} in the from of their summary and LinkedIn details.
Here's the information:

## Summary
{summary}

## LinkedIn profile
{linkedin}

With this context, please evaluate the latest response replying whether the response is acceptable and your feedback on this.
"""

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"""
    Here's the conversation between the User and the Agent: 
    {history}
    
    Here's the latest message for the User:
    {message}
    
    Here's the latest response from the Agent:
    {reply}
    
    Please evaluate the response replying whether it is acceptable and your feedback on this.
    """
    return user_prompt

In [23]:
import os

gpt_4o = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [30]:
def evaluate(reply, message, history) -> Evaluation | None:
    messages = [{
        "role": "system",
        "content": evaluator_system_prompt
    }] + history + [{
        "role": "user",
        "content": evaluator_user_prompt(reply, message, history)
    }]
    
    response = gpt_4o.beta.chat.completions.parse(
        model='gpt-4o',
        messages=messages,
        response_format=Evaluation
    )
    
    return response.choices[0].message.parsed

In [26]:
messages = [{
    "role": "system",
    "content": system_prompt
}] + [{
    "role": "user",
    "content": "do you hold a patent?"
}]
response = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

reply = response.choices[0].message.content

In [27]:
reply

"Yes, I hold a patent related to the problem of hiring engineers and the use of AI in that process. The invention focuses on an apparatus for determining role fitness while eliminating unwanted bias. This was a collaborative effort between my team at untapt and GQR, one of the world's fastest-growing recruitment firms. If you'd like to know more about it, feel free to ask!"

In [28]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It accurately addresses the user's question about holding a patent, providing details about the patent's focus and collaboration with GQR. The response is professional and engaging, offering to provide more information if desired, which aligns with Ed Donner's representation as a knowledgeable and open professional in the AI and recruitment domains.")

In [29]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = f"""
    {system_prompt}\n\n
    ## Previous answer rejected: You just tried to reply but the quality control rejected your response.
    
    ## Your attempted answer: 
    {reply}
    
    ## Reason for rejection: 
    {feedback}
    """
    
    messages = [{
        "role": "system",
        "content": updated_system_prompt
    }] + history + [{
        "role": "user",
        "content": message
    }]
    
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    
    return response.choices[0].message.content

In [33]:
def chat(message, history):
    system = system_prompt
    messages = [{
        "role": "system",
        "content": system
    }] + history + [{
        "role": "user",
        "content": message
    }]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    
    reply = response.choices[0].message.content
    
    evaluation = evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print(f"Evaluation passed")
    else:
        print(f"Evaluation failed - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
        
    return reply
        

In [ ]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Evaluation passed
Evaluation passed
